# Neural network based integrator
The code in this notebook can be used to train neutral network based integrators for Hamiltonian dynamical systems.

## Supported dynamical systems and integrators
Currently, two dynamical systems are implemented here:
* a simple **harmonic oscillator** with Hamiltonian $H(x,p) = \frac{p^2}{2m}+\frac{1}{2}k_{\text{spring}}x^2$. The phase space is two-dimensional ($d=2$) and the dynamics is linear. Set `system_label="harmonic oscillator"` to use this system.
* a system of **two coupled pendulums** suspended from ceiling and coupled with a spring. This is a non-linear system with a four-dimensional phase space ($d=4$). Set `system_label="coupled pendulums"` to use this system.

The systems can be integrated with two classes of neural network integrators:
1. an $s$-step method (set `use_hamiltonian = False`)
2. a symplectic integrator based on a Hamiltonian model (set `use_hamiltonian = True`)

For the $s$-step method the forward map is represented either by a forward map (set `use_LSTM=False`) or by an LSTM network as in [Kadupitiya et al. (2020)](https://arxiv.org/abs/2004.06493) (set `use_LSTM=True`).

For the symplectic integrator, either the Verlet method as described in [Greydanus et al. (2019)](https://arxiv.org/abs/1906.01563) can be used (set `use_strang_splitting=False`) or the higher-order Strang splitting for non-separable system as in [Tao (2016)](https://arxiv.org/abs/1609.02212) (set `use_strang_splitting=True`)

## Data generation
In all cases training data is generated by running the Verlet method with a very small timestep size $\Delta t_{\text{Verlet}}$. Since the exact solution is known for the Harmonic Oscillator, in this case also an exact integrator can be used (set `use_exact_integrator=True` in this case).

The data generator class in [data_generator.py](data_generator.py) is used to construct training data samples of the form $(X_j,y_j)$ where

$$
X_j = q_j^{(0)},q_j^{(\Delta t)},\dots,q_j^{((S-1)\Delta t)},\qquad\qquad y_j = q_j^{(S\Delta t)}.
$$

Here $q_j^{(0)}$ is a randomly chosen initial condition and the states $q_j^{(\Delta t)},q_j^{(2\Delta t)},\dots,q_j^{((S-1)\Delta t)}, q_j^{(S\Delta t)}$ are generated with a training generator (=Verlet) that is run with a smaller timestep $\Delta t_{\text{Verlet}}$.

The timestep size of the Neural network integrator is set to $\Delta t=F\Delta t_{\text{Verlet}}$ where $\Delta t_{\text{Verlet}}$ is the step size of the Verlet integrator that is used for training. For the multistep integrators we use $F=40$ and for the Hamiltonian integrator $F=20$ is used.


### Load required Python modules

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from dynamical_system import *
from time_integrator import *
from nn_integrator import *
from data_generator import *
from dynamic_timestep_training import *
%matplotlib inline

np.random.seed(2512517)

### Set up system
Set system parameters, construct dynamical system and integrator here. Adapt the values accordingly.

In [3]:
############## USER PARAMETERS ##############

# name of dynamical system to integrate
#system_label = 'harmonic oscillator'
system_label = 'coupled pendulums'

# use Hamiltonian model?
use_hamiltonian = True
# use LSTM network for multistep integrator?
use_LSTM = False
# use exact integrator for training?
use_exact_integrator=False
# Use Strang splitting?
use_strang_splitting=False

# timestep for training integrator
dt_train = 0.005
# number of steps for multistep neural network integrator
nsteps = 6

# Training hyperparameters
BATCH_SIZE=64
EPOCHS=1000
STEPS_PER_EPOCH=1000

#############################################

if system_label == 'harmonic oscillator':
    # mass of particle
    mass = 1.2
    # spring constant
    k_spring = 0.9
    dynamical_system = HarmonicOscillator(mass,k_spring)
elif system_label == 'coupled pendulums':
    # mass of particle
    mass = 1.0
    # length of rods
    L_rod = 1.0
    # distance of anchor points
    d_anchor = 1.0
    # spring constant
    k_spring = 1.0
    dynamical_system = CoupledPendulums(mass,L_rod,d_anchor,k_spring) 
else:
    print(f'ERROR: unknown dynamical system: \"{system_label}\"')

# Integrator used to generate data
if use_exact_integrator:
    train_integrator = ExactIntegrator(dynamical_system,dt_train)
else:
    if use_strang_splitting:
        train_integrator = StrangSplittingIntegrator(dynamical_system,dt_train)
    else:
        train_integrator = VerletIntegrator(dynamical_system,dt_train)


learning_rate = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-3,
    decay_steps=EPOCHS*STEPS_PER_EPOCH,
    alpha=1.E-3)

# Set neural network architecture
if use_hamiltonian:
    # timestep for neural network integrator
    dt = 0.1
    if use_strang_splitting:
        H_layers = [keras.layers.BatchNormalization(),
                    keras.layers.Dense(128,activation='tanh'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(64,activation='tanh'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(32,activation='tanh'),]
        nn_integrator = HamiltonianStrangSplittingNNIntegrator(dynamical_system,dt,H_layers,
                                                               learning_rate=learning_rate)
    else:
        V_pot_layers = [keras.layers.BatchNormalization(),
                        keras.layers.Dense(64,activation='tanh'),
                        keras.layers.BatchNormalization(),
                        keras.layers.Dense(64,activation='tanh'),
                        keras.layers.BatchNormalization(),
                        keras.layers.Dense(32,activation='tanh')]
        T_kin_layers = [keras.layers.Dense(64,activation='tanh'),
                        keras.layers.BatchNormalization(),
                        keras.layers.Dense(64,activation='tanh'),
                        keras.layers.BatchNormalization(),
                        keras.layers.Dense(32,activation='tanh')]
        nn_integrator = HamiltonianVerletNNIntegrator(dynamical_system,dt,V_pot_layers,T_kin_layers,
                                                      V_pot_layer_weights=None,T_kin_layer_weights=None,
                                                      learning_rate=learning_rate)

else:
    # timestep for neural network integrator
    dt = 0.2
    if use_LSTM: 
        # Use two layers of LSTMs followed by a dense layer
        dense_layers = [keras.layers.LSTM(64,return_sequences=True),
                        keras.layers.LSTM(64),
                        keras.layers.Dense(32,activation='tanh')]
    else:
        # Just use several dense layers
        dense_layers = [keras.layers.Flatten(),
                        keras.layers.Dense(32,activation='tanh'),
                        keras.layers.Dense(64,activation='tanh'),
                        keras.layers.Dense(32,activation='tanh')]


    nn_integrator = MultistepNNIntegrator(dynamical_system,dt,nsteps,
                                          dense_layers,
                                          learning_rate=learning_rate)

# visualise the neural network model
nn_integrator.model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 4)]            0         
                                                                 
 verlet_model (VerletModel)  (None, 4)                 13961     
                                                                 
Total params: 13,961
Trainable params: 13,445
Non-trainable params: 516
_________________________________________________________________


## Train neural network based integrator

Train the neural network integrator for the parameters chosen above.

In [ ]:
data_generator = DataGenerator(nn_integrator, train_integrator)

log_dir = './tb_logs/'
train_batches = data_generator.dataset.batch(BATCH_SIZE)
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

timestep_schedule = create_linear_timestep_schedule(dt_train=dt_train, dt_min=dt_train, dt_max=dt, nepoch=200, ninc=20)

dynamic_timestep_cb = DynamicTimestepCallback(data_generator,timestep_schedule,log_dir=log_dir)

result = nn_integrator.model.fit(train_batches,epochs=EPOCHS,steps_per_epoch=STEPS_PER_EPOCH,
                                 callbacks=[tensorboard_cb,dynamic_timestep_cb])